In [ ]:
!pip install transformers datasets faiss-cpu pandas torch

import pandas as pd
import torch
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
from transformers import MarianMTModel, MarianTokenizer


# CSV 파일에는 'question'과 'answer' 열이 있어야 합니다.
data = pd.read_csv('data.csv')

# 번역 모델 초기화 (영어 -> 한국어)
model_name = 'Helsinki-NLP/opus-mt-en-ko'
translator = MarianMTModel.from_pretrained(model_name)
tokenizer_translator = MarianTokenizer.from_pretrained(model_name)

# RAG 모델과 토크나이저 초기화
tokenizer = RagTokenizer.from_pretrained("facebook/rag-sequence-nq")
retriever = RagRetriever.from_pretrained("facebook/rag-sequence-nq")
model = RagSequenceForGeneration.from_pretrained("facebook/rag-sequence-nq")

# 질문에 대한 답변을 생성하는 함수
def answer_question(question):
    # 질문을 한국어로 번역
    translated = translate_to_korean(question)

    # RAG 모델을 통해 답변 생성
    input_ids = tokenizer(translated, return_tensors="pt").input_ids
    retrieved_docs = retriever(input_ids=input_ids, return_tensors="pt")

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids, 
            context_input_ids=retrieved_docs['context_input_ids'], 
            context_attention_mask=retrieved_docs['context_attention_mask']
        )

    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return answer[0]

# 영어 질문을 한국어로 번역하는 함수
def translate_to_korean(text):
    inputs = tokenizer_translator(text, return_tensors='pt', padding=True, truncation=True)
    translated = model.generate(**inputs)
    return tokenizer_translator.decode(translated[0], skip_special_tokens=True)

# 챗봇 인터페이스
def chat_with_bot():
    print("챗봇과 대화해 보세요! '종료'를 입력하면 대화가 종료됩니다.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == '종료':
            print("챗봇: 대화를 종료합니다.")
            break
        
        answer = answer_question(user_input)
        print(f"챗봇: {answer}\n")

# 챗봇 실행
chat_with_bot()
